In [1]:
import autogenerate
import importlib
importlib.reload(autogenerate)
autogenerate = autogenerate.autogenerate
import SimplexComplex as SC

In [2]:
def list_pair_to_array(lists, name, comment = None, type_name="int64_t"):
    r = {"name": name, "data": lists, "type": type_name}
    if comment is not None: r["comment"] = comment
    return r

In [3]:
def get_switch_data(sc):
    valid_indices = sc.valid_indices()
    all_to_valid = [-1 for _ in sc.all_tuples()]
    for i,v in enumerate(valid_indices):
        all_to_valid[v] = i
    return [
        list_pair_to_array(valid_indices,"auto_valid_tuple_indices", "For each valid tuple encodes the raw tuple index", type_name = "int8_t"),
        list_pair_to_array(all_to_valid,"auto_all_to_valid_tuple_indices", "Index of each tuple according to valid tuple indexing", type_name = "int8_t"),
list_pair_to_array(SC.valid_switch_table(sc),"auto_valid_switch_table", "Valid tuple indices", type_name = "int8_t"),
list_pair_to_array(SC.valid_switch_product_table(sc),"auto_valid_switch_product_table", "Tuple group product using valid tuple indices", type_name = "int8_t"),
list_pair_to_array(SC.valid_switch_inverse_table(sc),"auto_valid_switch_inverse_table", "Tuple group product inverse using valid tuple indices", type_name = "int8_t"),
           ]

In [4]:
import switch_tables_2d as st2d
sc2 = st2d.simplex_complex

In [5]:
tri_data = {"namespace": "tri_mesh", "tables": [
list_pair_to_array(st2d.edges,"auto_2d_edges", "lvids for a particular leid"),
list_pair_to_array(st2d.table(0,st2d.simplices),"auto_2d_table_vertex", "vertex switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(st2d.table(1,st2d.simplices),"auto_2d_table_edge", "edge switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(st2d.table_ccw(st2d.simplices),"auto_2d_table_ccw", "(tuple_index) -> is_ccw"),
list_pair_to_array(st2d.table_complete_tuple(0,st2d.simplices),"auto_2d_table_complete_vertex", "lvid -> a ccw [lvid,leid]"),
list_pair_to_array(st2d.table_complete_tuple(1,st2d.simplices),"auto_2d_table_complete_edge", "leid -> a ccw [lvid,leid]"),
] + get_switch_data(sc2)}



In [6]:
autogenerate("../src/wmtk/autogen/tri_mesh/autogenerated_tables.{}pp", tri_data, use_clang_format=True)


In [7]:
import switch_tables_3d as st3d
sc3 = st3d.simplex_complex

In [8]:
tet_data = {"namespace": "tet_mesh", "tables": [
list_pair_to_array(st3d.edges,"auto_3d_edges", "lvids for a specfic leid"),
list_pair_to_array(st3d.faces,"auto_3d_faces", "lvids for a specific lfid"),
list_pair_to_array(st3d.table(0,st3d.simplices),"auto_3d_table_vertex", "vertex switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table(1,st3d.simplices),"auto_3d_table_edge", "edge switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table(2,st3d.simplices),"auto_3d_table_face", "face switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table_ccw(st3d.simplices),"auto_3d_table_ccw", "(tuple_index) -> is_ccw"),
list_pair_to_array(st3d.table_complete_tuple(0,st3d.simplices),"auto_3d_table_complete_vertex", "lvid -> a ccw [lvid,leid,lfid]"),
list_pair_to_array(st3d.table_complete_tuple(1,st3d.simplices),"auto_3d_table_complete_edge", "evid -> a ccw [lvid,leid,lfid]"),
list_pair_to_array(st3d.table_complete_tuple(2,st3d.simplices),"auto_3d_table_complete_face", "fvid -> a ccw [lvid,leid,lfid]")]
            + get_switch_data(sc3)
           }

In [9]:
autogenerate("../src/wmtk/autogen/tet_mesh/autogenerated_tables.{}pp", tet_data, use_clang_format=True)
